In [ ]:
import requests
import pandas as pd
import os

"""
Lottery Results Fetching Script

This script fetches lottery results from an API, checks for new data, and saves results into a DataFrame.

Data Source: https://lotto.api.rayriffy.com
Author: Your Name
Date Created: YYYY-MM-DD
Last Updated: YYYY-MM-DD
"""

# Base URL
base_url = "https://lotto.api.rayriffy.com/list/"

# Column names in English
columns_english = ["Date", "1st Prize", "2nd Prize", "3rd Prize", "4th Prize", "5th Prize", "Front 3 Digits", "Back 3 Digits", "Back 2 Digits"]

# File path
file_path = "lottery_results.csv"

# Load existing data if available
if os.path.exists(file_path):
    existing_df = pd.read_csv(file_path, encoding="utf-8-sig")
    existing_dates = set(existing_df["Date"])
else:
    existing_df = pd.DataFrame(columns=columns_english)
    existing_dates = set()

print(f"Found {len(existing_dates)} existing records. Checking for updates...")

# Fetch all IDs from API
all_ids = []
page = 1
while True:
    response = requests.get(f"{base_url}{page}")

    if response.status_code == 200:
        data = response.json()
        if data["status"] == "success" and len(data["response"]) > 0:
            all_ids.extend([item["id"] for item in data["response"]])
            page += 1
        else:
            break
    else:
        print(f"Error fetching page {page}: Status code {response.status_code}")
        break

print(f"Total IDs collected: {len(all_ids)}")

# Collect lottery data not already in DataFrame
new_data = []

for lotto_id in all_ids:
    url = f"https://lotto.api.rayriffy.com/lotto/{lotto_id}"
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        if data["status"] == "success":
            date = data["response"].get("date", "N/A")

            if date in existing_dates:
                print(f"Skipping {date}, already exists.")
                continue

            print(f"Adding new data for {date}")

            prizes = data["response"].get("prizes", [])
            prize_first = ", ".join(prizes[0]["number"]) if len(prizes) > 0 else "N/A"
            prize_second = ", ".join(prizes[1]["number"]) if len(prizes) > 1 else "N/A"
            prize_third = ", ".join(prizes[2]["number"]) if len(prizes) > 2 else "N/A"
            prize_fourth = ", ".join(prizes[3]["number"]) if len(prizes) > 3 else "N/A"
            prize_fifth = ", ".join(prizes[4]["number"]) if len(prizes) > 4 else "N/A"

            running_numbers = data["response"].get("runningNumbers", [])
            front_three = ", ".join(running_numbers[0]["number"]) if len(running_numbers) > 0 else "N/A"
            back_three = ", ".join(running_numbers[1]["number"]) if len(running_numbers) > 1 else "N/A"
            back_two = ", ".join(running_numbers[2]["number"]) if len(running_numbers) > 2 else "N/A"

            new_data.append([date, prize_first, prize_second, prize_third, prize_fourth, prize_fifth, front_three, back_three, back_two])

    else:
        print(f"Error fetching Lotto ID {lotto_id}: Status code {response.status_code}")

if new_data:
    new_df = pd.DataFrame(new_data, columns=columns_english)
    final_df = pd.concat([existing_df, new_df], ignore_index=True)
    final_df.to_csv(file_path, encoding="utf-8-sig", index=False)
    print(f"Added {len(new_data)} new records to {file_path}")
else:
    print("No new data to update.")

In [13]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from statsmodels.tsa.arima.model import ARIMA

# โหลดข้อมูล
data = pd.read_csv('lottery_results.csv')

# เตรียมข้อมูล
data_clean = data[['Date', 'Back 2 Digits']].dropna()

def convert_thai_date(date_str):
    months = {
        'มกราคม': '01', 'กุมภาพันธ์': '02', 'มีนาคม': '03', 'เมษายน': '04',
        'พฤษภาคม': '05', 'มิถุนายน': '06', 'กรกฎาคม': '07', 'สิงหาคม': '08',
        'กันยายน': '09', 'ตุลาคม': '10', 'พฤศจิกายน': '11', 'ธันวาคม': '12'
    }
    parts = date_str.split()
    day = parts[0]
    month = months[parts[1]]
    year = str(int(parts[2]) - 543)
    return f'{year}-{month}-{int(day):02d}'

data_clean['Date'] = pd.to_datetime(data_clean['Date'].apply(convert_thai_date))
data_clean.sort_values('Date', inplace=True)

series = data_clean['Back 2 Digits'].values.reshape(-1, 1)
scaler = MinMaxScaler()
scaled_series = scaler.fit_transform(series)

# สร้าง supervised dataset
def create_dataset(dataset, look_back=10):
    X, Y = [], []
    for i in range(len(dataset) - look_back):
        X.append(dataset[i:(i + look_back), 0])
        Y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(Y)

look_back = 10
X, y = create_dataset(scaled_series, look_back)

# แบ่งชุดข้อมูล
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# ตรวจหาโมเดลที่ดีที่สุด
mae_scores = {
    'LSTM': mae_lstm,
    'RF': mae_rf,
    'ARIMA': mae_arima,
    'Ensemble': mae_ensemble
}
best_model = min(mae_scores, key=mae_scores.get)
accuracy = 100 - (mae_scores[best_model] / 99) * 100

# บันทึกเฉพาะผลลัพธ์ที่ทำนายได้จากโมเดลที่ดีที่สุด
predictions = {
    'LSTM': lstm_pred.flatten().round().astype(int).clip(0, 99),
    'RF': rf_pred.flatten().round().astype(int).clip(0, 99),
    'ARIMA': arima_pred.values.round().astype(int).clip(0, 99),
    'Ensemble': ensemble_predictions.round().astype(int).clip(0, 99)
}

best_predictions = pd.DataFrame({
    'Prediction': predictions[best_model]
})

best_predictions.to_csv(f'best_model_predictions_{best_model}.csv', index=False)

print(f'Best Model: {best_model}')
print(f'Accuracy: {accuracy:.2f}%')
print(f'Predictions saved to best_model_predictions_{best_model}.csv')

Best Model: ARIMA
Accuracy: 76.57%
Predictions saved to best_model_predictions_ARIMA.csv
